# VISUAL PIPELINE

In [1]:
import numpy as np
from scipy.stats import pearsonr, spearmanr
import sys
from PyQt5 import QtWidgets
import os
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import pickle
import glob
import datetime

from Running_computation import compute_speed
from Ca_imaging import CaImagingDataManager
from face_camera import FaceCamDataManager
from visual_stim import VisualStim
import General_functions
from visualization_GUI import VisualizationGUI
import Photodiode
from inputUI import InputWindow
import red_cell_function
import utils.file as file
from trial import Trial
import behavioral_states
import spontaneous as spont

## Constants

In [2]:
COMPILE = False

## Parameters

In [3]:
# paths
base_path = "C:/Users/mai-an.nguyen/Documents/16-00-59"
red_image_path = 'C:/Users/mai-an.nguyen/Documents/16-00-59/SingleImage-red/red.tif'
compile_dir = ''

# analysis settings
neuropil_impact_factor = 0.7
F0_method = 'sliding' #sliding or hamming
neuron_type = 'Other' #PYR or Other
starting_delay_2p = 0.100
num_samples = 1000

speed_threshold = 0.5
facemotion_threshold = 2
pupil_threshold = 2
pupil_threshold_type = 'std' #std or quantile

min_run_window = 3.5
min_as_window = 2.5
min_rest_window = 1.5

speed_filter_kernel = 10
motion_filter_kernel = 10
pupil_filter_kernel = 10
dFoF_filter_kernel = 10

## Load Data

In [4]:
# Get metadata
unique_id, global_protocol, experimenter, subject_id = file.get_metadata(base_path)
subject_id_anibio = file.get_mouse_id(base_path, subject_id)

No excel file for mouse metadata found : C:/Users/mai-an.nguyen/Documents/16-00-59\demo-Mouse.xlsx


In [5]:
# Load Ca-Imaging data
ca_img_dm = CaImagingDataManager(base_path, neuropil_impact_factor, F0_method, neuron_type, starting_delay_2p)
ca_img_dm.save_mean_image(base_path)
ca_img_dm.save_max_proj_image(base_path)
detected_roi = ca_img_dm._list_ROIs_idx
print('Original number of neurons :', len(detected_roi))

Mean_image_grayscale.png already exist.
Original number of neurons : 35


In [6]:
# Load Camera data
timestamp_start = Photodiode.get_timestamp_start(base_path)
face_cam_dm = FaceCamDataManager(base_path, timestamp_start)

C:/Users/mai-an.nguyen/Documents/16-00-59\FaceIt\FaceIt.npz doesn't exist.


In [7]:
# Compute speed
speed, speed_time_stamps = compute_speed(base_path)

In [8]:
# Resample facemotion, pupil and speed traces
if not face_cam_dm.no_face_data :
    last_F_index = np.argmin(np.abs(ca_img_dm.time_stamps - face_cam_dm.time_stamps[-1]))
    ca_img_dm.cut_frames(last_index=last_F_index) #update metrics with new frames length
new_time_stamps = ca_img_dm.time_stamps
total_duration = ca_img_dm.time_stamps[-1] - ca_img_dm.time_stamps[0]
print(f"Total duration of the recording: {total_duration} s")

#sub sampling and filtering speed
speed = General_functions.resample_signal(speed, 
                                          t_sample=speed_time_stamps, 
                                          new_freq=ca_img_dm.fs,
                                          interp_time=new_time_stamps,
                                          post_smoothing=2./50.)

if not face_cam_dm.no_face_data :
    pupil = General_functions.resample_signal(face_cam_dm.pupil, 
                                              t_sample=face_cam_dm.time_stamps, 
                                              new_freq=ca_img_dm.fs, 
                                              interp_time=new_time_stamps)
    facemotion = General_functions.resample_signal(face_cam_dm.facemotion, 
                                                   t_sample=face_cam_dm.time_stamps,
                                                   new_freq=ca_img_dm.fs, 
                                                   interp_time=new_time_stamps)

    # Normalize
    pupil = General_functions.scale_trace(pupil)
    facemotion = General_functions.scale_trace(facemotion)
else :
    facemotion, pupil = [np.nan] * len(new_time_stamps), [np.nan] * len(new_time_stamps)

Total duration of the recording: 2622.907376334 s


In [9]:
# Load Photodiode data
NIdaq, acq_freq = Photodiode.load_and_data_extraction(base_path)
Psignal_time, Psignal = General_functions.resample_signal(NIdaq['analog'][0],
                                                          original_freq=acq_freq,
                                                          new_freq=1000)

In [10]:
# Load Stimuli data
visual_stim = VisualStim(base_path)
protocol_df = visual_stim.protocol_df
protocol_df

                        name  duration
id                                    
0                moving-dots       3.0
1                random-dots       3.0
2               static-patch       1.0
3               looming-stim       5.0
4   Natural-Images-4-repeats       2.0
5                 grey-20min    1200.0
6          drifting-gratings       3.0


## Create saving folder

In [ ]:
# Create saving folder
save_dir, save_fig_dir, id_version = file.create_output_folder(base_path, unique_id)

""" id_version = '1'
save_dir = os.path.join(base_path, "_".join([unique_id, 'output', id_version]))
save_fig_dir = os.path.join(save_dir, "_".join([unique_id, 'figures'])) """

print(save_dir)
print(save_fig_dir)

C:/Users/mai-an.nguyen/Documents/16-00-59\2024_09_03_16-00-59_output_14
C:/Users/mai-an.nguyen/Documents/16-00-59\2024_09_03_16-00-59_output_14\2024_09_03_16-00-59_figures


## Calcium Imaging Pre-processing

In [12]:
# Detect ROIs with bad neuropils
ca_img_dm.detect_bad_neuropils()
kept2p_ROI = ca_img_dm._list_ROIs_idx
print('After removing bad neuropil neurons, nb of neurons :', len(kept2p_ROI))

After removing bad neuropil neurons, nb of neurons : 31


In [13]:
# Compute Fluorescence 
ca_img_dm.compute_F()
kept_ROI_alpha = ca_img_dm._list_ROIs_idx
print('Number of remaining neurons after alpha calculation :', len(kept_ROI_alpha))

Number of remaining neurons after alpha calculation : 31


In [14]:
# Calculation of F0
ca_img_dm.compute_F0(percentile=10, win=60)
kept_ROI_F0 = ca_img_dm._list_ROIs_idx
print('Number of remaining neurons after F0 calculation  :', len(kept_ROI_F0))

Number of remaining neurons after F0 calculation  : 30


In [15]:
# Calculation of dF over F0
ca_img_dm.compute_dFoF0()
computed_F_norm = ca_img_dm.normalize_time_series("dFoF0", lower=0, upper=3)

In [16]:
# Plot calcium imaging traces
ca_img_dm.plot('f0', sigma=0, mean=True, save_dir=save_fig_dir, legend=True)
ca_img_dm.plot('fluorescence', sigma=10, save_dir=save_fig_dir, legend=True)
ca_img_dm.plot('dFoF0', sigma=10, save_dir=save_fig_dir, legend=True)
ca_img_dm.plot_raster('fluorescence', sigma=10, save_dir=save_fig_dir)
ca_img_dm.plot_raster('dFoF0', sigma=10, save_dir=save_fig_dir)

## Correlation

In [17]:
speed_corr_list = [spearmanr(speed, ROI)[0] for ROI in ca_img_dm.dFoF0]
facemotion_corr_list = [spearmanr(facemotion, ROI)[0] for ROI in ca_img_dm.dFoF0]
pupil_corr_list = [spearmanr(pupil, ROI)[0] for ROI in ca_img_dm.dFoF0]

## Align times series

In [ ]:
# Set real stimuli onset with photodiode
visual_stim.realign_from_photodiode(Psignal_time, Psignal)
Psignal = General_functions.scale_trace(Psignal)

In [ ]:
# Stimuli start times and durations with calcium imaging data as a the basis
stim_time_end = list(visual_stim.real_time_onset + visual_stim.duration)
stim_time_period = [visual_stim.real_time_onset, stim_time_end]

F_Time_start_realigned, F_stim_init_indexes  = Photodiode.Find_F_stim_index(visual_stim.real_time_onset, ca_img_dm.time_stamps)

## Compute behavioral states

In [ ]:
# Calculate index boundaries of dark stimulus
idx_dark = np.argwhere(np.array(visual_stim.analyze_pupil) == 0)
idx_lim_dark, _ = visual_stim.get_protocol_onset_index(idx_dark, F_stim_init_indexes, ca_img_dm.fs)
idx_lim_dark

In [ ]:
min_states_window = {'run' : round(min_run_window * ca_img_dm.fs), 
                     'AS' : round(min_as_window * ca_img_dm.fs), 
                     'rest' : round(min_rest_window * ca_img_dm.fs)}

if not face_cam_dm.no_face_data :
    # Facemotion threshold
    real_time_states_facemotion, states_window_facemotion =\
        behavioral_states.split_stages(speed, facemotion, speed_threshold, facemotion_threshold, 
                                       ca_img_dm.time_stamps, min_states_window, ca_img_dm.fs, 
                                       'std', speed_filter_kernel, motion_filter_kernel)

    behavioral_states.stage_plot(speed, facemotion, pupil, ca_img_dm.dFoF0, 
                                 ca_img_dm.time_stamps, real_time_states_facemotion, states_window_facemotion, 
                                 save_fig_dir, speed_threshold, facemotion_threshold,'std', 'facemotion', 
                                 speed_filter_kernel, motion_filter_kernel, pupil_filter_kernel, dFoF_filter_kernel,
                                 svg=False)

    run_ratio_facemotion, as_ratio_facemotion, rest_ratio_facemotion =\
        behavioral_states.time_pie(real_time_states_facemotion, total_duration, 
                                   save_fig_dir,figname="states_duration_pie_facemotion")

    # Pupil threshold
    if len(idx_lim_dark) == 0 :
        real_time_states_pupil, states_window_pupil =\
            behavioral_states.split_stages(speed, pupil, speed_threshold, pupil_threshold, 
                                        ca_img_dm.time_stamps, min_states_window, ca_img_dm.fs, 
                                        pupil_threshold_type, speed_filter_kernel, pupil_filter_kernel)

        behavioral_states.stage_plot(speed, facemotion, pupil, ca_img_dm.dFoF0, 
                                    ca_img_dm.time_stamps, real_time_states_pupil, states_window_pupil, 
                                    save_fig_dir, speed_threshold, pupil_threshold, pupil_threshold_type, 'pupil', 
                                    speed_filter_kernel, motion_filter_kernel,  pupil_filter_kernel, dFoF_filter_kernel, 
                                    svg=False)
    else :
        real_time_states_pupil, states_window_pupil =\
            behavioral_states.split_stages_mixed(speed, pupil, facemotion, idx_lim_dark,
                                                 speed_threshold, pupil_threshold, facemotion_threshold,
                                                 ca_img_dm.time_stamps, min_states_window, ca_img_dm.fs, 
                                                 pupil_threshold_type, 'std',
                                                 speed_filter_kernel, pupil_filter_kernel, motion_filter_kernel)
        
        behavioral_states.stage_plot_mixed(speed, facemotion, pupil, ca_img_dm.dFoF0, 
                                            ca_img_dm.time_stamps, real_time_states_pupil, states_window_pupil, idx_lim_dark, ca_img_dm.fs,
                                            save_fig_dir, speed_threshold, pupil_threshold, facemotion_threshold, 
                                            pupil_threshold_type, 'std', 
                                            speed_filter_kernel, motion_filter_kernel,  pupil_filter_kernel, dFoF_filter_kernel, 
                                            svg=False)
    
    run_ratio_pupil, as_ratio_pupil, rest_ratio_pupil =\
        behavioral_states.time_pie(real_time_states_pupil, total_duration, 
                                   save_fig_dir, figname="states_duration_pie_pupil")
else :
    real_time_states, states_window =\
        behavioral_states.split_stages_locomotion(speed, speed_threshold, 
                                                  ca_img_dm.time_stamps, min_states_window, ca_img_dm.fs, speed_filter_kernel)

    behavioral_states.stage_plot_locomotion(speed, ca_img_dm.dFoF0, 
                                            ca_img_dm.time_stamps, real_time_states, states_window, 
                                            save_fig_dir, speed_threshold, speed_filter_kernel, 
                                            dFoF_filter_kernel,
                                            svg=False)

    run_ratio, rest_ratio =\
        behavioral_states.time_pie_locomotion(real_time_states, total_duration, 
                                   save_fig_dir,figname="states_duration_pie")

## Trial-averaging

In [ ]:
# Create Trial instance
trials = Trial(ca_img_dm, visual_stim, F_stim_init_indexes, attr='dFoF0', dt_pre_stim=1, dt_post_stim=1, auc_thr=5)

In [ ]:
# Compute responsive neurons
trials.find_responsive_rois(save_dir, folder_prefix="_".join([unique_id, id_version]))

# Save results in file
filename = "_".join([unique_id, id_version, 'protocol_validity_2'])
trials.save_protocol_validity(save_dir, filename)

In [ ]:
# Compute contextual modulation index
if "center-surround-cross" in visual_stim.protocol_names :
    cmi = trials.compute_cmi()
    trials.plot_cmi_hist(cmi, save_fig_dir)
    trials.plot_iso_vs_cross(save_fig_dir)
    surround_sup_cross, surround_sup_iso = trials.compute_surround_sup()
else :
    cmi = None
    surround_sup_cross, surround_sup_iso = None, None

In [23]:
# Compute the trial zscores not based on the averaged baseline but the baseline of the trace (used for plotting)
trial_zscores, pre_trial_zscores, post_trial_zscores = trials.compute_trial_zscores('dFoF0')

In [ ]:
# Plot trials related figures
for i in range(len(protocol_df)):    
    if visual_stim.stim_cat[i] :

        #plot trial-averaged z-score raster sorted
        trials.trial_average_rasterplot(i, savepath=save_fig_dir) 

        #plot trial-averaged z-score raster not sorted
        trials.trial_average_rasterplot(i, savepath=save_fig_dir, sort=False)

        #plot trial-averaged z-score raster sorted and normalized
        trials.trial_average_rasterplot(i, savepath=save_fig_dir, normalize=True)
        
        #plot trials z-score raster with paired baseline
        trials.trial_rasterplot(trial_zscores, pre_trial_zscores, post_trial_zscores, i, 'dFoF0', savepath=save_fig_dir)
        
        #plot trials z-score raster with averaged baseline
        #trials.trial_rasterplot(trials.trial_zscores, trials.pre_trial_zscores, trials.post_trial_zscores, i, trials.ca_attr, savepath=save_fig_dir)
        
        for k in range(len(ca_img_dm._list_ROIs_idx)):
            #plot trial-averaged z-score traces
            trials.plot_stim_response(i, k, save_dir, folder_prefix="_".join([unique_id, id_version]))

            #plot trial-averaged normalized with baseline traces
            trials.plot_norm_trials(i, k, save_dir, folder_prefix="_".join([unique_id, id_version]))

In [24]:
# Behavioral states of mouse during trials

if not face_cam_dm.no_face_data :
    real_time_states_sorted = behavioral_states.sort_dict_el(real_time_states_pupil)
else :
    real_time_states_sorted = behavioral_states.sort_dict_el(real_time_states)

# Sort trials by arousal states (output in trials.npy)
trials.sort_trials_by_states(F_Time_start_realigned, real_time_states_sorted)

# Compute averaged traces by arousal states
trials.compute_avg_by_states()

for i in range(len(protocol_df)):    
    if visual_stim.stim_cat[i] :
        
        #plot trials with behavioral states
        trials.plot_stim_occurence(i, trial_zscores, pre_trial_zscores, real_time_states_sorted, F_Time_start_realigned, save_dir, folder_prefix="_".join([unique_id, id_version]))
        
        for k in range(len(ca_img_dm._list_ROIs_idx)):
            #plot trial-averaged traces per arousal states
            trials.plot_trials_per_states(i, k, save_dir, folder_prefix="_".join([unique_id, id_version]))

### Other Method : Bootstrapping Method

In [ ]:
# Method : Bootstrapping
protocol_validity = []
for protocol in range(len(protocol_df)):
    chosen_protocol = protocol_df.index[protocol]
    protocol_duration = protocol_df['duration'][protocol]
    protocol_name = protocol_df['name'][protocol]
    protocol_validity_i = Photodiode.average_image(ca_img_dm.dFoF0, visual_stim.order, chosen_protocol,protocol_duration, protocol_name, F_stim_init_indexes, ca_img_dm.fs, num_samples, save_dir, file_prefix="_".join([unique_id, id_version]))
    protocol_validity.append(protocol_validity_i)

filename_protocol = "_".join([unique_id, id_version, 'protocol_validity']) + ".npz"
np.savez(os.path.join(save_dir, filename_protocol), **{key: value for d in protocol_validity for key, value in d.items()})
print(protocol_validity)

## Spontaneous behavior

In [ ]:
spont_stimuli_id, _ = spont.get_spont_stim(visual_stim)
sigma = 5

if len(spont_stimuli_id) > 0 :
    spont_speed_corr_list = []
    spont_facemotion_corr_list = []
    spont_pupil_corr_list = []
    valid_neurons_speed_list = []
    valid_neurons_facemotion_list = []
    valid_neurons_pupil_list = []
    
    idx_lim_protocol, spont_stimuli_id_order, F_spontaneous = visual_stim.get_protocol_onset_index(spont_stimuli_id, F_stim_init_indexes, ca_img_dm.fs, tseries=ca_img_dm.dFoF0)

    spont_df = protocol_df.loc[spont_stimuli_id_order]

    for i, id  in zip(range(len(spont_stimuli_id_order)), spont_stimuli_id_order):
        [start_spont_index, end_spont_index] = idx_lim_protocol[i]
        name_stimuli = spont_df.loc[spont_stimuli_id_order[i]]['name']
        save_spont_dir = os.path.join(save_dir, "_".join([unique_id, 'spontaneous']))
        save_spont_dir_i = os.path.join(save_spont_dir, name_stimuli)
        if not os.path.exists(save_spont_dir_i) : os.makedirs(save_spont_dir_i)

        time_stamps_spont = new_time_stamps[start_spont_index:end_spont_index]
        print(f"Spontaneous activity time {name_stimuli} {i}: from {time_stamps_spont[0]} s to {time_stamps_spont[-1]} s")

        # Speed correlation
        speed_spont = speed[start_spont_index:end_spont_index]
        spont_speed_corr, valid_neurons_temp = spont.compute_spont_corr(speed_spont, F_spontaneous[i], time_stamps_spont, sigma, 'speed', save_spont_dir_i)
        spont_speed_corr_list.append(spont_speed_corr)
        valid_neurons_speed_list.append(valid_neurons_temp)
        spont.colormap_perm_test(time_stamps_spont, F_spontaneous[i], speed_spont, valid_neurons_temp, spont_speed_corr, sigma=sigma, label='speed', save_path=save_spont_dir_i)

        if not face_cam_dm.no_face_data :
            # Facemotion correlation
            facemotion_spont = facemotion[start_spont_index:end_spont_index]
            spont_facemotion_corr, valid_neurons_temp = spont.compute_spont_corr(facemotion_spont, F_spontaneous[i], time_stamps_spont, sigma, 'facemotion', save_spont_dir_i)
            spont_facemotion_corr_list.append(spont_facemotion_corr)
            valid_neurons_facemotion_list.append(valid_neurons_temp)
            spont.colormap_perm_test(time_stamps_spont, F_spontaneous[i], facemotion_spont, valid_neurons_temp, spont_facemotion_corr, sigma=sigma, label='facemotion', save_path=save_spont_dir_i)

            # Pupil correlation
            pupil_spont = pupil[start_spont_index:end_spont_index]
            if spont_df.loc[id].analyze_pupil :
                spont_pupil_corr, valid_neurons_temp = spont.compute_spont_corr(pupil_spont, F_spontaneous[i], time_stamps_spont, sigma, 'pupil', save_spont_dir_i)
                spont_pupil_corr_list.append(spont_pupil_corr)
                valid_neurons_pupil_list.append(valid_neurons_temp)
                spont.colormap_perm_test(time_stamps_spont, F_spontaneous[i], pupil_spont, valid_neurons_temp, spont_pupil_corr, sigma=sigma, label='pupil', save_path=save_spont_dir_i)

    speed_corr = np.dot(np.array(spont_speed_corr_list).T, spont_df.duration/np.sum(spont_df.duration))
    valid_neurons_speed = spont.get_valid_neurons(valid_neurons_speed_list)
    spont.pie_plot(len(valid_neurons_speed), len(ca_img_dm._list_ROIs_idx) - len(valid_neurons_speed), save_spont_dir, 'speed')
    valid_neurons_speed_list2 = np.zeros((len(ca_img_dm._list_ROIs_idx), len(valid_neurons_speed_list)))
    for i in range(len(valid_neurons_speed_list)) : 
        valid_neurons_speed_list2[valid_neurons_speed_list[i], i] = 1

    if not face_cam_dm.no_face_data :
        facemotion_corr = np.dot(np.array(spont_facemotion_corr_list).T, spont_df.duration/np.sum(spont_df.duration))
        valid_neurons_facemotion = spont.get_valid_neurons(valid_neurons_facemotion_list)
        spont.pie_plot(len(valid_neurons_facemotion), len(ca_img_dm._list_ROIs_idx) - len(valid_neurons_facemotion), save_spont_dir, 'facemotion')
        valid_neurons_facemotion_list2 = np.zeros((len(ca_img_dm._list_ROIs_idx), len(valid_neurons_facemotion_list)))
        for i in range(len(valid_neurons_facemotion_list)) : 
            valid_neurons_facemotion_list2[valid_neurons_facemotion_list[i], i] = 1

        if len(spont_pupil_corr_list) > 0 :
            spont_dt_pupil = spont_df[spont_df.analyze_pupil == 1].duration
            pupil_corr = np.dot(np.array(spont_pupil_corr_list).T, spont_dt_pupil/np.sum(spont_dt_pupil))
            valid_neurons_pupil = spont.get_valid_neurons(valid_neurons_pupil_list)
            spont.pie_plot(len(valid_neurons_pupil), len(ca_img_dm._list_ROIs_idx) - len(valid_neurons_pupil), save_spont_dir, 'pupil')
            valid_neurons_pupil_list2 = np.zeros((len(ca_img_dm._list_ROIs_idx), len(valid_neurons_pupil_list)))
            for i in range(len(valid_neurons_pupil_list)) : 
                valid_neurons_pupil_list2[valid_neurons_pupil_list[i], i] = 1
        else :
            nb_rois = len(ca_img_dm._list_ROIs_idx)
            nan_array = np.empty(nb_rois)
            nan_array.fill(np.nan)
            pupil_corr = nan_array
            valid_neurons_pupil_list2 = []
    
    else :
        nb_rois = len(ca_img_dm._list_ROIs_idx)
        nan_array = np.empty(nb_rois)
        nan_array.fill(np.nan)
        facemotion_corr, pupil_corr = nan_array, nan_array
        valid_neurons_facemotion_list2, valid_neurons_pupil_list2 = [], []
        
else : 
    speed_corr, facemotion_corr, pupil_corr = np.array(speed_corr_list), np.array(facemotion_corr_list), np.array(pupil_corr_list)
    valid_neurons_speed_list2, valid_neurons_facemotion_list2, valid_neurons_pupil_list2 = None, None, None

## Set variables for saving and GUI

In [24]:
# for saving file
photodiode = (Psignal_time, Psignal)
pupilAndTimeSt  = (new_time_stamps, pupil)
fmotionAndTimeSt  = (new_time_stamps, facemotion)
speedAndTimeSt = (new_time_stamps, speed)
background_image_path = os.path.join(base_path, "Mean_image_grayscale.png")
filename_protocol = "_".join([unique_id, id_version, 'protocol_validity_2']) + ".npz"
protocol_validity_npz = np.load(os.path.join(save_dir, filename_protocol), allow_pickle=True)

## Save ouptut files

### Visual stimuli info in excel sheet

In [12]:
filename = "_".join([unique_id, id_version, 'visual_stim_info']) + ".xlsx"
visual_stim.export_df_to_excel(save_dir, filename)

### HDF5 file

In [25]:
filename = "_".join([unique_id, id_version, 'postprocessing']) + ".h5"
H5_dir = os.path.join(save_dir, filename)
hf = h5py.File(H5_dir, 'w')
behavioral_group = hf.create_group('Behavioral')
correlation = behavioral_group.create_group("Correlation")
spont_correlation = behavioral_group.create_group("Spont_correlation")
caImg_group = hf.create_group('Ca_imaging')
caImg_full_trace = caImg_group.create_group('full_trace')
stimuli_group = hf.create_group("Stimuli")
rois_group = hf.create_group("ROIs")
states_group = hf.create_group("Arousal states")
if not face_cam_dm.no_face_data :
    states_with_pupil = states_group.create_group("Arousal states pupil")
    frame_bounds_pupil = states_with_pupil.create_group("Frame bounds")
    time_bounds_pupil = states_with_pupil.create_group("Time bounds")
    states_with_facemotion = states_group.create_group("Arousal states facemotion")
    frame_bounds_facemotion = states_with_facemotion.create_group("Frame bounds")
    time_bounds_facemotion = states_with_facemotion.create_group("Time bounds")
else :
    frame_bounds = states_group.create_group("Frame bounds")
    time_bounds = states_group.create_group("Time bounds")

file.create_H5_dataset(behavioral_group, [speedAndTimeSt, fmotionAndTimeSt, pupilAndTimeSt, photodiode], ['Speed', 'FaceMotion', 'Pupil', 'Photodiode'])
file.create_H5_dataset(correlation, [speed_corr_list, facemotion_corr_list, pupil_corr_list], ['speed_corr', 'facemotion_corr', 'pupil_corr'])
if len(spont_stimuli_id) > 0 :
    file.create_H5_dataset(spont_correlation, [spont_speed_corr_list, spont_facemotion_corr_list, spont_pupil_corr_list], ['speed_corr', 'facemotion_corr', 'pupil_corr'])
    spont_valid_rois = spont_correlation.create_group("Valid_ROIs")
    file.create_H5_dataset(spont_valid_rois, [valid_neurons_speed_list2, valid_neurons_facemotion_list2, valid_neurons_pupil_list2], ['speed', 'facemotion', 'pupil'])
caImg_group.create_dataset('Time', data=ca_img_dm.time_stamps)
file.create_H5_dataset(caImg_full_trace, [ca_img_dm.raw_F, ca_img_dm.raw_Fneu, ca_img_dm.fluorescence, ca_img_dm.f0, ca_img_dm.dFoF0], 
                                    ['raw_F', 'raw_Fneu', 'F', 'F0', 'dFoF0'])
file.create_H5_dataset(stimuli_group, [visual_stim.real_time_onset, F_Time_start_realigned, F_stim_init_indexes], 
                                    ['time_onset', 'time_onset_caimg_timescale', 'idx_onset_caimg_timescale'])
if cmi is not None :
    file.create_H5_dataset(stimuli_group, [cmi, surround_sup_cross, surround_sup_iso], ['cmi', 'surround_sup_cross', 'surround_sup_iso'])
file.create_H5_dataset(rois_group, [detected_roi, kept2p_ROI, kept_ROI_alpha, kept_ROI_F0], 
                                    ['0_original', '1_neuropil', '2_alpha', '3_F0'])
if not face_cam_dm.no_face_data :
    file.create_H5_dataset(frame_bounds_pupil, [states_window_pupil['run'], states_window_pupil['AS'], states_window_pupil['rest']], ['Run', 'AS', 'Rest'])
    file.create_H5_dataset(time_bounds_pupil, [real_time_states_pupil['run'], real_time_states_pupil['AS'], real_time_states_pupil['rest']], ['Run', 'AS', 'Rest'])
    file.create_H5_dataset(frame_bounds_facemotion, [states_window_facemotion['run'], states_window_facemotion['AS'], states_window_facemotion['rest']], ['Run', 'AS', 'Rest'])
    file.create_H5_dataset(time_bounds_facemotion, [real_time_states_facemotion['run'], real_time_states_facemotion['AS'], real_time_states_facemotion['rest']], ['Run', 'AS', 'Rest'])
else :
    file.create_H5_dataset(frame_bounds, [states_window['run'], states_window['rest']], ['Run', 'Rest'])
    file.create_H5_dataset(time_bounds, [real_time_states['run'], real_time_states['rest']], ['Run', 'Rest'])

hf.close()

### Trial data

In [ ]:
filename = "_".join([unique_id, id_version, 'trials'])
trials.save_trials(save_dir, filename)

### Calcium imaging stat.npy file

In [26]:
filename = "_".join([unique_id, id_version, 'stat.npy'])
np.save(os.path.join(save_dir, filename), ca_img_dm.stat, allow_pickle=True)

### Compilation excel sheet

In [27]:
if COMPILE :
    data_df = pd.DataFrame({
                "Session_id": unique_id, "Output_id": id_version, "Protocol": global_protocol, "Experimenter": experimenter, "Mouse_id": subject_id_anibio,
                'Mean_speed' : np.nanmean(speed), 'Std_speed' : np.nanstd(speed),
                'Mean_fmotion' : np.nanmean(facemotion), 'Std_fmotion' : np.nanstd(facemotion),
                'Mean_pupil' : np.nanmean(pupil), 'Std_pupil' : np.nanstd(pupil),
                'Spontaneous' : True if len(spont_stimuli_id) > 0 else False,
                'Mean_speed_corr' : np.nanmean(speed_corr), 
                'Mean_fmotion_corr' : np.nanmean(facemotion_corr),
                'Mean_pupil_corr' : np.nanmean(pupil_corr), 
                'Mean_dFoF0' : np.nanmean(ca_img_dm.dFoF0), 
                'Run % (pupil)' : run_ratio_pupil if not face_cam_dm.no_face_data else None, 
                'AS % (pupil)' : as_ratio_pupil if not face_cam_dm.no_face_data else None,
                'Rest % (pupil)' : rest_ratio_pupil if not face_cam_dm.no_face_data else None,
                'Run % (motion)' : run_ratio_facemotion if not face_cam_dm.no_face_data else None, 
                'AS % (motion)' : as_ratio_facemotion if not face_cam_dm.no_face_data else None, 
                'Rest % (motion)' : rest_ratio_facemotion if not face_cam_dm.no_face_data else None,
                'Run %' : run_ratio if face_cam_dm.no_face_data else None, 
                'Rest %' : rest_ratio if face_cam_dm.no_face_data else None, 
                }, index=[0]).set_index("Session_id")
    file.compile_xlsx_file(data_df, compile_dir)

### Analysis settings

In [28]:
settings = {"Date" : datetime.date.today(),
            "Time" : datetime.datetime.now().time(),
            "Session_id" : unique_id,
            "Neuron type" : neuron_type,
            "Neuropil impact factor" : ca_img_dm._neuropil_if,
            "F0 calculateion method" : F0_method,
            "2p starting delay" : starting_delay_2p,
            "Bootstrapping nb of samples" : num_samples,
            "Speed threshold" : f"{speed_threshold} (cm/s)",
            "Facemotion threshold" : f"{facemotion_threshold} (std)",
            "Pupil threshold" :  f"{pupil_threshold} ({pupil_threshold_type})",
            "Minimum running window" : min_run_window,
            "Minimum AS window" : min_as_window,
            "Minimum rest window" : min_rest_window,
            "Speed filter kernel" : speed_filter_kernel,
            "Motion filter kernel" : motion_filter_kernel,
            "Pupil filter kernel" : pupil_filter_kernel,
            "Fluorescence filter kernel" : dFoF_filter_kernel,
            "Analyzed folder" : base_path,
            "Saving folder" : save_dir,
            "Compile folder" : compile_dir
            }
file.save_analysis_settings(settings, save_dir)

## Launch visualization GUI

In [ ]:
app = QtWidgets.QApplication(sys.argv)
main_window = VisualizationGUI(save_dir, 
                               ca_img_dm.stat, ca_img_dm.ops, background_image_path,
                               protocol_validity_npz, 
                               speed_corr, facemotion_corr, pupil_corr, 
                               computed_F_norm, ca_img_dm.time_stamps, speedAndTimeSt, fmotionAndTimeSt, pupilAndTimeSt, photodiode, stim_time_period, 
                               red_image_path)
main_window.show()
app.exec_()